# Many with astrocytes

Let $A_i$ denote an $N_z$ by $N_z$ matrix encoding the synaptic connections controlled by astrocyte $i$.

$$
\begin{eqnarray*}
    \dot{z_i} & = & z_i ( (\lambda_i^{(z)} + \mathrm{i}) + b_i^{(z)} |z_i|^2) + \sum_k W_{ik} z_k \\
    \dot{a_i} & = & a_i ( (\lambda_i^{(a)} + \mathrm{i}) + b_i^{(a)} |a_i|^2) + \theta \left(\left(\frac{1}{N_a} \sum_k a_k\right) - a_i\right) + \varphi \left( \sum_k \left(\sum_l (A_i)_{lk}\right) |z_k| \right) \\
    \dot{W} & = & \alpha\left( I_{N_z} - \mathbf{z} \bar{\mathbf{z}}^\top \right) - \beta\left( \sum_k |a_k| A_k \right)
\end{eqnarray*}
$$

In [1]:
import numpy as np
import scipy.integrate
import matplotlib.pyplot as plt

In [2]:
Nz = 20
Na = 20

# Hopf oscillator characteristics
lz = np.random.randn( Nz )
la = np.random.randn( Na ) - 4
# TODO Should the Lyopunov exponent vary across oscillators?
bz = -0.1 + np.random.randn( Nz ) * 1.j
ba = -0.1 + 0.1 * np.random.randn( Nz ) * 1.j

alpha = 0.001 # 0.1
beta = 0.01

theta = 0.01
phi = 0.01

In [14]:
A = []
for i in range( Na ):
    A.append( np.random.randn( Nz, Nz ) )

In [15]:
def deriv( t, y ):
    
    z = y[:Nz]
    a = y[Nz:(Nz+Nz)]
    W = np.reshape( y[(Nz+Na):], (Nz, Nz) )
    
    z_dot = np.zeros( z.shape[0], dtype = np.complex )
    for i in range( z_dot.shape[0] ):
        z_dot[i] = z[i] * ( ( lz[i] + 1.j ) + bz[i] * z[i] * np.conj( z[i] ) )
        for k in range( z_dot.shape[0] ):
#             if k == i:
#                 continue
#             z_dot[i] += W[i, k] * np.real( z[k] )
            z_dot[i] += W[i, k] * z[k]
    
    a_dot = np.zeros( a.shape[0], dtype = np.complex )
    a_bar = (1. / a.shape[0]) * np.sum( a )
    for i in range( a_dot.shape[0] ):
        a_dot[i] = a[i] * ( ( la[i] + 1.j ) + ba[i] * a[i] * np.conj( a[i] ) ) + theta * (a_bar - a[i])
        for k in range( z_dot.shape[0] ):
            a_dot[i] += phi * np.sum( A[i][:, k] ) * np.abs( z[k] )
    
#     W_dot = alpha * ( np.eye( Nz ) - np.outer( np.real( z ), np.real( z ) ) )
    W_dot = alpha * ( np.eye( Nz ) - np.outer( z, np.conj( z ) ) )
    for k in range( a_dot.shape[0] ):
        W_dot += beta * np.abs( a[k] ) * A[k]
    
    y_dot = np.zeros( y.shape[0], dtype = np.complex )
    y_dot[:Nz] = z_dot
    y_dot[Nz:(Nz+Na)] = a_dot
    y_dot[(Nz+Na):] = W_dot.flatten()
    
    return y_dot

In [16]:
t_span = [0, 2e4]
t_eval = np.arange( t_span[0], t_span[-1], 1e-2 )

In [17]:
# TODO Randomize the z0 in an intelligent way
z0 = np.zeros( (Nz,), dtype = np.complex )
for i in range( z0.shape[0] ):
    z0[i] = 0.01 + 0.j
    
# TODO Randomize the a0 in an intelligent way
a0 = np.zeros( (Na,), dtype = np.complex )
for i in range( a0.shape[0] ):
    a0[i] = 0.01 + 0.j
    
W0 = np.random.randn( Nz, Nz ) + np.random.randn( Nz, Nz ) * 1.j

In [18]:
y0 = np.zeros( (Nz + Na + Nz*Nz,), dtype = np.complex )
y0[:Nz] = z0
y0[Nz:(Nz+Na)] = a0
y0[(Nz+Na):] = W0.flatten()

In [19]:
sol = scipy.integrate.solve_ivp( deriv, t_span, y0,
                                 t_eval = t_eval )

KeyboardInterrupt: 

In [ ]:
t_star = sol.t
y_star = sol.y

z_star = y_star[:Nz, :]
a_star = y_star[Nz:(Nz+Na), :]
W_star = np.reshape( y_star[(Nz+Na):, :], (Nz, Nz, y_star.shape[1]) )

In [ ]:
plt.figure( figsize = (48, 6) )
plt.imshow( np.real( a_star ), aspect = 'auto',
            vmin = np.quantile( np.real( a_star ), 0.01 ),
            vmax = np.quantile( np.real( a_star ), 0.99 ) )

In [ ]:
plt.figure( figsize = (48, 6) )
plt.imshow( np.real( z_star ), aspect = 'auto',
            vmin = np.quantile( np.real( z_star ), 0.01 ),
            vmax = np.quantile( np.real( z_star ), 0.99 ) )